# 距離学習
入力：OSNetから得た外観特徴量ベクトル512次元  
出力：圧縮した外観特徴量ベクトル128次元  
損失関数：triplet loss  

訓練データ，検証データ，テストデータは以下の３通り．各データセットにおいて以下のコードを繰り返す  

- 訓練データ，検証データ：撮影実験１，２　，テストデータ：撮影実験３
- 訓練データ，検証データ：撮影実験２，３　，テストデータ：撮影実験１
- 訓練データ，検証データ：撮影実験３，１　，テストデータ：撮影実験２

## ライブラリとパラメーター

- 訓練：検証：テスト = 6：2：2としている．（訓練データ，検証データとテストデータは事前に分けているので本当は訓練：検証 = 8:2でいいはず．このコードでは訓練データを全て使い切れていない．要修正）
- 距離定義　コサイン類似度


In [1]:
""" ライブラリ読込 """
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import Subset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier

from pytorch_metric_learning.miners import TripletMarginMiner
from pytorch_metric_learning.distances import CosineSimilarity
from pytorch_metric_learning.losses import TripletMarginLoss
from pytorch_metric_learning.reducers import ThresholdReducer
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

from pytorch_metric_learning import losses, miners, distances, reducers, samplers
from pytorch_metric_learning import losses, distances, regularizers

from umap import UMAP

# データ作成のパラメーター
TRAIN_TEST_RATIO = 0.2
TRAIN_VAL_RATIO = 0.2
exclusion_num = 99

# 学習のパラメーター
epochs = 500
learning_rate = 1e-4
batch_size = 128
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# デフォルトのファイル保存先, 名前
save_path = "/content/"
file_name = "動作確認"

# TripletMarginLoss
distance = CosineSimilarity()
reducer = ThresholdReducer(low = 0)
loss_func = TripletMarginLoss(margin=0.2, distance=distance, reducer=reducer)
mining_func = TripletMarginMiner(margin=0.2, distance=distance, type_of_triplets="semi-hard")

ModuleNotFoundError: No module named 'pandas'

## データ作成関数

全ての関数を使用しているわけではない．
特に今回の研究では，全て一括のコードで実行しているわけではない．以下の関数を使用して事前に違うコードにて，テストデータからオーギュメンテーション加工を消したりしている．（全てのコードを公開するのは疲れるので，載せません）


In [ ]:
def split_time(df):
  df_exp1 = df[df['file_name'].str.contains('exp1')]
  df_exp2 = df[df['file_name'].str.contains('exp2')]
  df_exp3 = df[df['file_name'].str.contains('exp3')]
  return df_exp1, df_exp2, df_exp3


def split_point(df):
  df_point1 = df[df['file_name'].str.contains('point1')]
  df_point2 = df[df['file_name'].str.contains('point2')]
  df_point3 = df[df['file_name'].str.contains('point3')]
  df_point4 = df[df['file_name'].str.contains('point4')]
  df_point5 = df[df['file_name'].str.contains('point5')]
  df_point6 = df[df['file_name'].str.contains('point6')]
  return df_point1, df_point2, df_point3, df_point4, df_point5, df_point6

def split_person(df):
  df_others = df[df['label'] == 0]
  df_ichimanda = df[df['label'] == 1]
  df_ichiraku = df[df['label'] == 2]
  df_idogawa = df[df['label'] == 3]
  df_miyazaki = df[df['label'] == 4]
  df_noguchi = df[df['label'] == 5]
  df_sakai = df[df['label'] == 6]
  return df_others, df_ichimanda, df_ichiraku, df_idogawa, df_miyazaki, df_noguchi, df_sakai


In [3]:
def del_augmented_data_choice(df, del_aug_list):
  # DataFrameのbool値の反転には「~」を使う　https://itips.krsw.biz/pandas-dataframe-how-to-reverse-bool-column/
  aug_dict = {"blur":'_0.jpg', "bright":'_1.jpg', "contrast":'_３.jpg', "contrast_flip":'_7.jpg'}
  for aug in del_aug_list:
    df = df[~df['file_name'].str.contains(aug_dict[aug])]
  return df


def del_augmented_data_all(df):
  # テストのデータを元データのみにする（テストデータに水増しにが入るのはおかしいから）
  # DataFrameのbool値の反転には「~」を使う　https://itips.krsw.biz/pandas-dataframe-how-to-reverse-bool-column/
  aug_list = ['_0.jpg', '_1.jpg', '_3.jpg' , '_7.jpg']
  for aug in aug_list:
    df = df[~df['file_name'].str.contains(aug)]
  return df


def del_duplication_data(df_train, df_test):
  file_name_list = df_test['file_name'].tolist()
  for i, file_name in enumerate(file_name_list):
    df_train = df_train[~df_train['file_name'].str.contains(file_name)]
    print(i)
  return df_train


In [ ]:
学習データにおいて，すべてのラベルのサンプル数が同数になるように，min_randam_samplingを用いて，サンプル数が最小のラベルに合わせてランダムサンプリングをする

その後，

In [ ]:
def min_randam_sampling(df):
  # dfを最小のラベルの個数に合わせてそれぞれランダムサンプリング
  number = df['label'].value_counts().min()
  filename_list = df['label'].unique()
  df_temp = pd.DataFrame()
  for i in filename_list:
    df_p = df.groupby("label").get_group(i).sample(n=number)
    # 結合
    df_temp = pd.concat([df_temp, df_p])
  df = df_temp
  df.shape
  print("最小のラベルの個数に合わせてそれぞれランダムサンプリングしました")
  return df


def del_exclusion_num_person(df, exclusion_num):
  # 条件に応じて一部のデータを除外する
  #99はその他を含む 0はその他だけ除外, 1以降はその他と番号の被験者を除外
  exclusion_num = exclusion_num
  number = df['label'].value_counts().min()
  filename_list = df['label'].unique()


  if exclusion_num == 99:
    print("その他の人を含むデータセットです")

    # label:0を削減
    number = 5000
    df_s = df.groupby("label").get_group(0).sample(n=number)
    df_t = df[df['label'] != 0]
    # 結合
    df_sample = pd.concat([df_s, df_t])
    df_sample

  elif exclusion_num == 0:
    print("その他の人を抜いたデータセットです")
    df_sample = df[df['label'] != 0]
    df_sample["label"] = df_sample["label"] -1

  else:
    print("label"+str(exclusion_num)+"の人を抜いたデータセットです")
    for i in filename_list:
      if i == 0:
        continue
      elif i < exclusion_num:
        df_p = df.groupby("label").get_group(i).sample(n=number)
        df_p["label"] = df_p["label"] -1
        df_sample = pd.concat([df_sample, df_p])
      elif i == exclusion_num:
        continue
      elif i > exclusion_num:
        df_p = df.groupby("label").get_group(i).sample(n=number)
        df_p["label"] = df_p["label"] -2
        df_sample = pd.concat([df_sample, df_p])


  df = df_sample
  print(df.iloc[:, 0].value_counts(sort=False))
  return df, exclusion_num


In [ ]:

def make_train_test(df):
  ### 訓練からテストを切り出す場合
  df_train = df
  df_train, df_test = train_test_split(df_train, test_size=TRAIN_TEST_RATIO, random_state=0)

  df_train.columns = df.columns.values
  df_test.columns = df.columns.values
  return df_train, df_test


def make_X_y(df_train, df_test):
  ### 訓練とテストを別々に用意する場合

  # 特徴量 (説明変数) 訓練
  X_train = df_train.iloc[:, 3:]
  # 特徴量 (説明変数)テスト
  X_test = df_test.iloc[:, 3:]
  # ラベル（目的変数)
  y_train = df_train.iloc[:, 0]
  # ラベル（目的変数)
  y_test = df_test.iloc[:, 0]

  return X_train, X_test, y_train, y_test


def make_dataloader(X_train, X_test, y_train, y_test):
  print("----------------------")
  X_train,X_val,y_train,y_val = train_test_split(X_train, y_train, test_size=TRAIN_VAL_RATIO, random_state=0, stratify=y_train)


  print("----------------------")
  print("trainに含まれる各labelの個数")
  print(y_train.value_counts(sort=True))
  print("----------------------")
  print("valに含まれる各labelの個数")
  print(y_val.value_counts(sort=True))
  print("----------------------")
  print("testに含まれる各labelの個数")
  print(y_test.value_counts(sort=True))



GPU上でpytorchを用いる場合は，Pytorch tensorに変換する

In [ ]:
 '''PyTorch tensorへ変換'''
  # https://megane-man666.hatenablog.com/entry/pytorch
  # PyTorchのデータセットは、numpy型を取り込んで、tensor型でする必要があります。
  # そこで、.valuesを使ってDataFrame型からnumpy.ndarray型に変換して、torch.Tesor()、LongTensor()でtensor型に変換しています。
  # 説明変数をTensor()、目的変数をLongTensor()で変換しないと、RuntimeErrorが発生しますので間違えないようにしましょう。
  X_train = torch.Tensor(X_train.values)
  X_val = torch.Tensor(X_val.values)
  X_test = torch.Tensor(X_test.values)
  y_train = torch.LongTensor(y_train.values)
  y_val = torch.LongTensor(y_val.values)
  y_test = torch.LongTensor(y_test.values)

  train_data = TensorDataset(X_train, y_train)
  val_data = TensorDataset(X_train, y_train)
  test_data = TensorDataset(X_test, y_test)

  # https://kdl-di.hatenablog.com/entry/2021/12/24/090000
  # 一般的に機械学習では、学習用データ、検証用データ、評価用データの三種類を要します。
  # 学習用データ：機械学習モデルが学習する際に利用するデータ。複数回利用する
  # 検証用データ：学習データで学習したモデルが、どの程度汎化性能を持っているか評価する際に利用するデータ。複数回利用する
  # 評価用データ：学習したモデルが未知のデータに対してどれだけ適応できるかを検証する。学習したモデルに対して一度のみ利用する

  train_loader = DataLoader(train_data, batch_size, shuffle=True)
  val_loader = DataLoader(val_data, batch_size)
  test_loader = DataLoader(test_data)

  return train_loader, val_loader, test_loader


In [ ]:
def show_file_name(df):
  print(str(df) + "に含まれる画像の名前です")
  for i in df["file_name"]:
    print(i)

def show_label_count(df):
  print("testに含まれる各labelの個数")
  print(df.iloc[:, 0].value_counts(sort=True))


# モデルの定義
class Net(nn.Module):

    # ユニット・層の数・活性化関数等ニューラルネットワークの模型となるものを下記に記述
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(512, 400)
        self.fc2 = nn.Linear(400, 300)
        self.bn1 = nn.BatchNorm1d(300)
        self.fc3 = nn.Linear(300, 200)
        self.dropout_1 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(200, 100)
        self.dropout_2 = nn.Dropout(0.2)
        self.fc5 = nn.Linear(100, 50)
        self.bn2 = nn.BatchNorm1d(50)
        self.fc6 = nn.Linear(50, 25)
        self.bn3 = nn.BatchNorm1d(25)
        self.fc7 = nn.Linear(25, 20)
        self.fc8 = nn.Linear(20, 2)

    # 順方向の計算処理の流れを下記に記述
    def forward(self, x):
        x = F.relu(self.fc1(x))  # ReLU: max(x, 0)
        x = F.relu(self.fc2(x))
        x = self.bn1(x)
        x = F.relu(self.fc3(x))
        x = self.dropout_1(x)
        x = F.relu(self.fc4(x))
        x = self.dropout_2(x)
        x = F.relu(self.fc5(x))
        x = self.bn2(x)
        x = F.relu(self.fc6(x))
        x = self.bn3(x)
        x = F.relu(self.fc7(x))
        x = self.fc8(x)
        return x



# 学習用関数

In [ ]:
# 学習用関数
def train(model, loss_func, mining_func, device, dataloader, optimizer, epoch):
    model.train()
    total_loss = 0
    for idx, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(inputs)
        indices_tuple = mining_func(embeddings, labels)
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    #     if idx % 10 == 0:
    #         print('Epoch {} Iteration {}: Loss = {}, Number of mined triplets = {}'.format(epoch, idx, loss, mining_func.num_triplets))
    # print()
    return total_loss


# バリデーション用関数
def val(model, dataloader, device, epoch):
    model.eval()
    with torch.no_grad():
        total_loss = 0
        for i, (inputs, labels) in enumerate(dataloader):
            inputs, labels = inputs.to(device), labels.to(device)
            embeddings = model(inputs)
            indices_tuple = mining_func(embeddings, labels)
            loss = loss_func(embeddings, labels, indices_tuple)
            loss
            total_loss += loss.item()

    return total_loss


# テスト用関数
def test(model, dataloader, device):
    predicted_metrics = []
    true_labels = []
    model.eval()
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloader):
            inputs, labels = inputs.to(device), labels.to(device)
            metric = model(inputs).detach().cpu().numpy()
            metric = metric.reshape(metric.shape[0], metric.shape[1])
            predicted_metrics.append(metric)
            true_labels.append(labels.detach().cpu().numpy())
    return np.concatenate(predicted_metrics), np.concatenate(true_labels)



# 学習用関数

### 実際に使用した関数
- run_PML_128
- train()
- val()
- read_csv()
- make_exp1_aug()　（名前が悪い！！exp1もaugも関係なし．特徴量データを訓練．検証，テストに分ける関数）
- make_dataloder()

結局使用している関数は少ないね…

In [ ]:
### 学習実行する関数

def run_PML(save_path, file_name):
    model = Net().to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 学習と評価
    train_loss_list = []
    val_loss_list = []

    train_acc_list = []
    val_acc_list = []

    best_loss = None
    best_loss_epoch = 0

    # val_lossとtrain_loss両方で判定(train_lossが十分に下がっていないのに学習終了するのを防ぐ)
    for epoch in range(1, epochs + 1):
        print('Epoch {}/{}'.format(epoch, epochs))
        # print('-' * 10)
        train_loss = train(model, loss_func, mining_func, device, train_loader, optimizer, epoch)
        val_loss = val(model, val_loader, device, epoch)
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        ## best_lossより小さなval_lossが出力された場合のみモデルを保存する。##
        if best_loss is None:
            best_loss = val_loss

        ## val_loss最小が更新されたらモデルを保存
        elif val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), './best_model_PML_exp1.pth')
            print(str(best_loss_epoch) + "エポック目で最小のval_loss:" + str(best_loss) + "を更新したため、モデルを保存しました")
            best_loss_epoch = epoch + 1

        ## best_lossが0になったら終了。##
        elif (best_loss == 0) and (train_loss < 0.5):
            torch.save(model.state_dict(), './best_model_PML_exp1.pth')
            best_loss_epoch = epoch + 1
            break

    print(str(best_loss_epoch - 1) + "エポック目が最小のval_loss=" + str(best_loss) + "でした")

    # # loss_historyを表示して保存
    show_loss_history(train_loss_list, val_loss_list, save_path, file_name)

    # # モデルの出力が2次元のとき
    df_extract_train_PML = extract_feature_PML(train_loader)
    df_extract_test_PML = extract_feature_PML(test_loader)
    show_PML_output(df_extract_train_PML, df_extract_test_PML, save_path, file_name)

    return df_extract_train_PML, df_extract_test_PML

    # モデルの出力が多次元のとき
    # 特徴量抽出
    df_extract_train_PML = extract_feature_PML(train_loader)
    df_extract_test_PML = extract_feature_PML(test_loader)

    # UMAPで2次元に圧縮
    df_extract_train_PML_UMAP, df_extract_test_PML_UMAP = apply_UMAP(df_extract_train_PML, df_extract_test_PML)

    # 表示
    show_PML_output(df_extract_train_PML_UMAP, df_extract_test_PML_UMAP, save_path, file_name)

    return df_extract_train_PML_UMAP, df_extract_test_PML_UMAP### 学習実行する関数

def run_PML(save_path, file_name):
    model = Net().to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 学習と評価
    train_loss_list = []
    val_loss_list = []

    train_acc_list = []
    val_acc_list = []

    best_loss = None
    best_loss_epoch = 0

    # val_lossとtrain_loss両方で判定(train_lossが十分に下がっていないのに学習終了するのを防ぐ)
    for epoch in range(1, epochs + 1):
        print('Epoch {}/{}'.format(epoch, epochs))
        # print('-' * 10)
        train_loss = train(model, loss_func, mining_func, device, train_loader, optimizer, epoch)
        val_loss = val(model, val_loader, device, epoch)
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        ## best_lossより小さなval_lossが出力された場合のみモデルを保存する。##
        if best_loss is None:
            best_loss = val_loss

        ## val_loss最小が更新されたらモデルを保存
        elif val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), './best_model_PML_exp1.pth')
            print(str(best_loss_epoch) + "エポック目で最小のval_loss:" + str(best_loss) + "を更新したため、モデルを保存しました")
            best_loss_epoch = epoch + 1

        ## best_lossが0になったら終了。##
        elif (best_loss == 0) and (train_loss < 0.5):
            torch.save(model.state_dict(), './best_model_PML_exp1.pth')
            best_loss_epoch = epoch + 1
            break

    print(str(best_loss_epoch - 1) + "エポック目が最小のval_loss=" + str(best_loss) + "でした")

    # # loss_historyを表示して保存
    show_loss_history(train_loss_list, val_loss_list, save_path, file_name)

    # # モデルの出力が2次元のとき
    df_extract_train_PML = extract_feature_PML(train_loader)
    df_extract_test_PML = extract_feature_PML(test_loader)
    show_PML_output(df_extract_train_PML, df_extract_test_PML, save_path, file_name)

    return df_extract_train_PML, df_extract_test_PML

    # モデルの出力が多次元のとき
    # 特徴量抽出
    df_extract_train_PML = extract_feature_PML(train_loader)
    df_extract_test_PML = extract_feature_PML(test_loader)

    # UMAPで2次元に圧縮
    df_extract_train_PML_UMAP, df_extract_test_PML_UMAP = apply_UMAP(df_extract_train_PML, df_extract_test_PML)

    # 表示
    show_PML_output(df_extract_train_PML_UMAP, df_extract_test_PML_UMAP, save_path, file_name)

    return df_extract_train_PML_UMAP, df_extract_test_PML_UMAP


class Net128(nn.Module):
    def __init__(self):
        super(Net128, self).__init__()
        self.fc1 = nn.Linear(512, 384)
        self.fc2 = nn.Linear(384, 288)  # 調整
        self.bn1 = nn.BatchNorm1d(288)  # 調整
        self.fc3 = nn.Linear(288, 224)  # 調整
        self.dropout_1 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(224, 160)  # 調整
        self.dropout_2 = nn.Dropout(0.2)
        self.fc5 = nn.Linear(160, 128)  # 調整
        self.bn2 = nn.BatchNorm1d(128)  # 調整
        self.fc6 = nn.Linear(128, 128)  # 最終層に近づくため同じ次元数を保持

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.bn1(x)
        x = F.relu(self.fc3(x))
        x = self.dropout_1(x)
        x = F.relu(self.fc4(x))
        x = self.dropout_2(x)
        x = F.relu(self.fc5(x))
        x = self.bn2(x)
        x = self.fc6(x)  # 最終層
        return x
##############################################################################
######################################################################
### 学習実行する関数(モデル学習のみ）

def run_PML128(save_path, file_name):
    model = Net128().to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 学習と評価
    train_loss_list = []
    val_loss_list = []

    train_acc_list = []
    val_acc_list = []

    best_loss = None
    best_loss_epoch = 0

    # val_lossとtrain_loss両方で判定(train_lossが十分に下がっていないのに学習終了するのを防ぐ)
    for epoch in range(1, epochs + 1):
        print('Epoch {}/{}'.format(epoch, epochs))
        # print('-' * 10)
        train_loss = train(model, loss_func, mining_func, device, train_loader, optimizer, epoch)
        val_loss = val(model, val_loader, device, epoch)
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        ## best_lossより小さなval_lossが出力された場合のみモデルを保存する。##
        if best_loss is None:
            best_loss = val_loss

        ## val_loss最小が更新されたらモデルを保存
        elif val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), './best_model_PML128_point5_kouzumi.pth')
            print(str(best_loss_epoch) + "エポック目で最小のval_loss:" + str(best_loss) + "を更新したため、モデルを保存しました")
            best_loss_epoch = epoch + 1

        ## best_lossが0になったら終了。##
        elif (best_loss == 0) and (train_loss < 0.5):
            torch.save(model.state_dict(), './best_model_PML128_point5_koizumi.pth')
            best_loss_epoch = epoch + 1
            break

    print(str(best_loss_epoch - 1) + "エポック目が最小のval_loss=" + str(best_loss) + "でした")

    return best_loss_epoch - 1, best_loss


def run_logistic_regression(X_train, X_test, y_train, y_test, save_path, file_name):
    lr = LogisticRegression()  # ロジスティック回帰モデルのインスタンスを作成
    lr.fit(X_train, y_train)  # ロジスティック回帰モデルの重みを学習

    Y_pred = lr.predict(X_test)
    df_report_LR = show_report(y_test, Y_pred, save_path, file_name)
    return df_report_LR


### 精度検証関数

def extract_feature_PML(data_loader):
    # モデルの再定義
    model = Net().to(device)
    # 学習済みモデルの読み出し
    model.load_state_dict(torch.load('./best_model_PML.pth'))
    model.eval()

    metric, label = test(model, data_loader, device)

    metrics = pd.DataFrame(metric)
    label = pd.DataFrame(label)

    df_extract = pd.concat([label, metrics], axis=1)

    return df_extract


def extract_feature_DNN(data_loader):
    # モデルの再定義
    model = Net().to(device)
    # 学習済みモデルの読み出し
    model.load_state_dict(torch.load('./best_model_DNN.pth'))
    model.eval()

    df_extract = pd.DataFrame()
    for i, (data, label) in enumerate(data_loader):
        data = data.to(device)
        label = label.numpy().tolist()

        features = model(data)
        label_df = pd.Series(label)

        features_np = features.to('cpu').detach().numpy().copy()
        features_df = pd.DataFrame(features_np)

        features_df = pd.concat([label_df, features_df], axis=1)

        df_extract = df_extract.append(features_df)

    pd.options.display.float_format = '{:.2f}'.format
    df_extract = df_extract.reset_index(drop=True)
    print(df_extract)
    return df_extract


######################################################################

def apply_UMAP(df_extract_train, df_extract_test):
    # 　高次元の特徴量を教師ありUMAPで2次元にして返す関数

    UMAP_X_train = df_extract_train.iloc[:, 1:]
    UMAP_X_test = df_extract_test.iloc[:, 1:]
    UMAP_Y_train = df_extract_train.iloc[:, 0]
    UMAP_Y_test = df_extract_test.iloc[:, 0]

    # UMAP
    # 訓練とテストを同じ空間に投射したいときは、fit_transformとtransformを使い分ける
    # https://aotamasaki.hatenablog.com/entry/2018/07/28/220102

    umap = UMAP(n_components=2, random_state=0, n_neighbors=100, metric="cosine")
    embedding_train = umap.fit_transform(UMAP_X_train, UMAP_Y_train)  # 教師あり
    # embedding_train = umap.fit_transform(UMAP_X_train) # 教師なし
    embedding_test = umap.transform(UMAP_X_test)

    embedding_train = pd.DataFrame(embedding_train)
    embedding_test = pd.DataFrame(embedding_test)

    df_extract_train = pd.concat([UMAP_Y_train, embedding_train], axis=1)
    df_extract_test = pd.concat([UMAP_Y_test, embedding_test], axis=1)

    return df_extract_train, df_extract_test


# 深層学習モデルで使った訓練データを使ってknnを訓練、テストデータを投入して予測ラベルを出力
def apply_kNN(df_extract_train, df_extract_test):
    kNN_X_train = df_extract_train.iloc[:, 1:]
    kNN_X_test = df_extract_test.iloc[:, 1:]
    kNN_Y_train = df_extract_train.iloc[:, 0]
    kNN_Y_test = df_extract_test.iloc[:, 0]

    kNN = KNeighborsClassifier(n_neighbors=5)
    kNN.fit(kNN_X_train, kNN_Y_train)

    kNN_Y_pred = kNN.predict(kNN_X_test)

    return kNN_Y_test, kNN_Y_pred


######################################################################

def show_loss_history(train_loss_list, val_loss_list, save_path, file_name):
    fig_loss_history = plt.figure(figsize=(20, 10))
    plt.plot(train_loss_list, alpha=0.8, label="Train Loss")
    plt.plot(val_loss_list, alpha=0.8, label="Val Loss")
    plt.grid()
    plt.legend()
    plt.show()
    fig_loss_history.savefig(save_path + file_name + "_" + "loss_history")


def show_2D_metric(df_extract, title, save_path, file_name):
    # 散布図
    df_label = df_extract.iloc[:, 0]
    num_list = np.sort(df_label.unique())

    # matplotlibでのラベルの付け方
    # https://blog.amedama.jp/entry/umap
    fig_2D_metric = plt.figure(figsize=(10, 10))
    for n in num_list:
        plt.scatter(df_extract[df_extract.iloc[:, 0] == n].iloc[:, 1],
                    df_extract[df_extract.iloc[:, 0] == n].iloc[:, 2],
                    label=n, alpha=0.8)

    plt.title(title)
    plt.legend()
    plt.show()
    fig_2D_metric.savefig(save_path + file_name + "_" + title)


def show_report(label_list, pred_list, save_path, file_name):
    # 機械学習の分類評価指標 混同行列と正解率・適合率・再現率・F値
    # https://laid-back-scientist.com/confusion-matrix

    # 【sklearn】Classification_reportの使い方を丁寧に
    # https://gotutiyan.hatenablog.com/entry/2020/09/09/111840

    # 混同行列を作成
    cm = confusion_matrix(label_list, pred_list)
    # https://evaluelog.com/post-122/
    index = ["others", "ichimanda", "ichiraku", "idogawa", "miyazaki", "noguchi", "sakai"]
    columns = ["others", "ichimanda", "ichiraku", "idogawa", "miyazaki", "noguchi", "sakai"]
    if exclusion_num == 0:
        del index[0]  # othersを削除
        del columns[0]  # othersを削除
    elif exclusion_num == 99:
        pass
    elif exclusion_num != 0:
        del index[0]  # othersを削除
        del columns[0]  # othersを削除
        del index[exclusion_num - 1]  # 指定した番号の人を除外(上でothersを削除したので1ずれる)
        del columns[exclusion_num - 1]  # 指定した番号の人を除外(上でothersを削除したので1ずれる)
    cm = pd.DataFrame(data=cm, index=index, columns=columns)

    # 混同行列を可視化
    # Seabornでheatmapを表示させた時の数字を1.5e+02→150に変更[Python]
    # https://onl.bz/zMYfsFy
    plt.rcParams["figure.figsize"] = (12, 10)
    glaph = sns.heatmap(cm, annot=True, cmap="Blues", fmt='d')  # 「fmt='d'」をつける
    glaph.set(xlabel="predict", ylabel="label")
    # 画像で保存
    fig = glaph.get_figure()
    fig.savefig(save_path + file_name + "_" + "cm" + ".png")

    # Classification_reportを表示
    pd.options.display.float_format = '{:.4g}'.format
    eval_dict = classification_report(label_list, pred_list, target_names=columns, output_dict=True)
    df_report = pd.DataFrame(eval_dict)  # DataFrameとして表示
    # csvで保存
    df_report.to_csv(save_path + file_name + "_" + "report" + ".csv")

    print(df_report)

    return df_report


def show_PML_output(df_extract_train_PML, df_extract_test_PML, save_path, file_name):
    # 訓練データを入力したときの出力を可視化
    show_2D_metric(df_extract_train_PML, "PML-train-metric", save_path, file_name)
    # テストデータを入力したときの出力を可視化
    show_2D_metric(df_extract_test_PML, "PML-test-metric", save_path, file_name)

    # 精度評価(2次元ベクトルにkNN)
    kNN_Y_test_PML, kNN_Y_pred_PML = apply_kNN(df_extract_train_PML, df_extract_test_PML)
    df_report_PML = show_report(kNN_Y_test_PML, kNN_Y_pred_PML, save_path, file_name)
    df_report_PML


def show_DNN_output(df_extract_train_PML, df_extract_test_PML, save_path, file_name):
    # 訓練データを入力したときの出力を可視化
    show_2D_metric(df_extract_train_PML, "PML-train-metric", save_path, file_name)
    # テストデータを入力したときの出力を可視化
    show_2D_metric(df_extract_test_PML, "PML-test-metric", save_path, file_name)

    # 精度評価(2次元ベクトルにkNN)
    kNN_Y_test_PML, kNN_Y_pred_PML = apply_kNN(df_extract_train_PML, df_extract_test_PML)
    df_report_PML = show_report(kNN_Y_test_PML, kNN_Y_pred_PML, save_path, file_name)
    df_report_PML


# appの読み込み
def read_csv(file_list, label0=True, normalize=True):
    count_file = 0
    for file in file_list:
        if count_file == 0:
            count_file += 1
            # メモリ不足を防ぐために分割して読み込み
            data_reader = pd.read_csv(file, chunksize=500)
            df_app = pd.concat((r for r in data_reader), ignore_index=True)
            print("----------------------")
            print("1つ目のデータ")
            print(df_app.iloc[:, 0].value_counts(sort=False))

        else:
            # メモリ不足を防ぐために分割して読み込み
            count_file += 1
            data_reader = pd.read_csv(file, chunksize=500)
            df_file = pd.concat((r for r in data_reader), ignore_index=True)
            print("----------------------")
            print(str(count_file) + "つ目のデータ")
            print(df_file.iloc[:, 0].value_counts(sort=False))
            df_app = pd.concat([df_app, df_file])

    if normalize:
        df_p = df_app.iloc[:, 0:3]
        df_q = df_app.iloc[:, 3:]
        df_q = (df_q - df_q.min()) / (df_q.max() - df_q.min())

        df_app = pd.concat([df_p, df_q], axis=1)
        df_app = df_app.fillna(0)

    if label0:
        # label:0を削減
        number = 5000
        df_s = df_app.groupby("label").get_group(0).sample(n=number)
        df_t = df_app[df_app['label'] != 0]
        # 結合
        df_app = pd.concat([df_s, df_t])

    print("----------------------")
    print("appの合計のデータ")
    print(df_app.iloc[:, 0].value_counts(sort=False))
    print("----------------------")
    print("最小のlabel数")
    print(df_app['label'].value_counts().min())

    return df_app



# 学習実行部

file_listには，訓練データのみを使用する．（全体データから，テストデータとなる撮影実験のサンプルを削除するとよい）

In [ ]:
# 全身のオリジナル＋ブラー+明るさ+コントラスト

file_list = ["C:/Users/sugie/PycharmProjects/pythonProject10/orient_tranceformed_data/filltered_feature_all_orient5_num_metric_with_point5_koizumi.csv"]
df_app = read_csv(file_list)

# 実行に時間がかかるので注意

def make_exp1_aug(df_app):
    ### 実験1
    # 訓練:exp123/地点123456/被験者oabcdef
    # テスト:exp123/地点123456/被験者oabcdef

    df_train = df_app

    # 最小サンプル数に合わせる
    df_train = min_randam_sampling(df_train)

    # 訓練とテストに分割
    df_train, df_test = train_test_split(df_train, test_size=TRAIN_TEST_RATIO, random_state=0)

    # 訓練の中から、テスト画像のaug版を削除する(訓練とテストの実質的な重複を防ぐ)
    df_train = del_duplication_data(df_train, df_test)

    # 特徴量 (説明変数) 訓練
    X_train = df_train.iloc[:, 3:]
    # ラベル（目的変数)
    y_train = df_train.iloc[:, 0]

    # X_yに分割
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=TRAIN_TEST_RATIO, random_state=0,
                                                        stratify=y_train)

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = make_exp1_aug(df_app)
train_loader, val_loader, test_loader = make_dataloader(X_train, X_test, y_train, y_test)

for i in range(1, 6):
    print("make_exp1_aug 関数を呼び出し中...")
    X_train, X_test, y_train, y_test = make_exp1_aug(df_app)
    print("make_dataloader 関数を呼び出し中...")
    train_loader, val_loader, test_loader = make_dataloader(X_train, X_test, y_train, y_test)
    print("run_PML 関数を呼び出し中...")
    run_PML128("C:/Users/sugie/PycharmProjects/pythonProject10/モデル保存８", "実験1-"+str(i))
    print()
    print(str(i) + "回目が終了しました")
    print("###############################################################")
